In [7]:
from pathlib import Path
import pandas as pd
import os
import sys
import importlib
sys.path.append(os.path.abspath(".."))

import src.config
importlib.reload(src.config) 
import src.data_loader
importlib.reload(src.data_loader) 
import src.features
importlib.reload(src.features) 
import src.eda
importlib.reload(src.eda) 
import src.utils.plotting
importlib.reload(src.utils.plotting) 

from src.config import Settings
from src.data_loader import DataLoader
from src.features import FeatureEngineer
from src.eda import EDAAnalyzer
from src.utils.plotting import Plotter

cfg = Settings()
cfg

dl = DataLoader(cfg)

# Try to download fresh data. If no internet, skip quietly if files already exist.
try:
    _ = dl.fetch_and_cache()
    print("Downloaded fresh data.")
except Exception as e:
    print("Download failed or skipped:", e)
    missing = [t for t in cfg.tickers if not (cfg.data_raw_dir / f"{t}.csv").exists()]
    if missing:
        raise FileNotFoundError(f"Missing cached raw CSVs for: {missing}. Connect to internet and re-run fetch.")
    else:
        print("Using cached CSVs in data/raw/")

# If your DataLoader has load_all_list() use it; else fallback builds the list
try:
    frames = dl.load_all_list()
except AttributeError:
    frames = [df.assign(Ticker=t) for t, df in dl.load_all().items()]

fe = FeatureEngineer(cfg)
features = fe.pipeline(frames)  # saves to data/processed/merged_features.csv

# Extra safety: coerce numeric & fill any residual gaps
import pandas as pd
features = features.apply(pd.to_numeric, errors="coerce").ffill().bfill()

display(features.dtypes.head())
display(features.tail())
print("Date range:", features.index.min(), "→", features.index.max())
print("Saved dataset:", cfg.data_processed_dir / "merged_features.csv")

pl = Plotter(cfg.reports_figures_dir)

price_cols = ["TSLA","BND","SPY"]
ret_cols   = ["TSLA_ret","BND_ret","SPY_ret"]

p1 = pl.line(features, price_cols, "Adj Close Prices", "closing_prices.png")
p2 = pl.line(features, ret_cols,   "Daily Returns",     "daily_returns.png")

print("Saved figures:")
print(" -", p1)
print(" -", p2)


eda = EDAAnalyzer()

tsla_ret = features["TSLA_ret"].dropna()
roll = eda.rolling_stats(tsla_ret, windows=(21,63,252))
p3 = pl.line(roll, roll.columns.tolist(), "TSLA rolling mean/std (returns)", "tsla_rolling_stats.png")

basic = eda.basic_stats(features, price_cols + ret_cols)
display(basic)

# Save a CSV snapshot for your interim report
out_csv = (cfg.reports_figures_dir.parent / "interim" / "basic_stats.csv")
out_csv.parent.mkdir(parents=True, exist_ok=True)
basic.to_csv(out_csv)

print("Saved:")
print(" -", p3)
print(" -", out_csv)

adf_prices  = eda.adf_test(features["TSLA"])          # likely non-stationary
adf_returns = eda.adf_test(features["TSLA_logret"])   # more likely stationary

print("ADF — TSLA (price):", adf_prices)
print("ADF — TSLA (log returns):", adf_returns)

summary = {}
for t in ["TSLA","BND","SPY"]:
    r = features[f"{t}_ret"].dropna()
    summary[t] = {
        "VaR_95_daily": eda.var_95(r),
        "Sharpe_annual": eda.sharpe(r, rf_annual=cfg.risk_free_rate),
        "Annual_return_est": r.mean() * 252,
        "Annual_vol_est": r.std() * (252 ** 0.5),
    }

import pandas as pd
risk_df = pd.DataFrame(summary).T
display(risk_df)

risk_csv = (cfg.reports_figures_dir.parent / "interim" / "risk_metrics.csv")
risk_df.to_csv(risk_csv)
print("Saved risk metrics CSV:", risk_csv)

saved = [
    cfg.data_processed_dir / "merged_features.csv",
    cfg.reports_figures_dir / "closing_prices.png",
    cfg.reports_figures_dir / "daily_returns.png",
    cfg.reports_figures_dir / "tsla_rolling_stats.png",
    cfg.reports_figures_dir.parent / "interim" / "basic_stats.csv",
    cfg.reports_figures_dir.parent / "interim" / "risk_metrics.csv",
]
print("Artifacts saved:")
for p in saved:
    print(" -", p)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded fresh data.


Ticker
BND        float64
SPY        float64
TSLA       float64
BND_ret    float64
SPY_ret    float64
dtype: object

Ticker,BND,SPY,TSLA,BND_ret,SPY_ret,TSLA_ret,BND_logret,SPY_logret,TSLA_logret
Date,,,,,,,,,
2025-07-24,72.698860,634.419983,305.299988,-0.001232,0.000331,-0.081970,-0.001233,0.000331,-0.085525
2025-07-25,72.878265,637.099976,316.059998,0.002468,0.004224,0.035244,0.002465,0.004215,0.034637
2025-07-28,72.758659,636.940002,325.589996,-0.001641,-0.000251,0.030152,-0.001643,-0.000251,0.029707
2025-07-29,73.137398,635.260010,321.200012,0.005205,-0.002638,-0.013483,0.005192,-0.002641,-0.013575
2025-07-30,72.948036,634.460022,319.040009,-0.002589,-0.001259,-0.006725,-0.002592,-0.001260,-0.006748


Date range: 2015-07-02 00:00:00 → 2025-07-30 00:00:00
Saved dataset: ..\data\processed\merged_features.csv
Saved figures:
 - ..\reports\figures\closing_prices.png
 - ..\reports\figures\daily_returns.png


Ticker,TSLA,BND,SPY,TSLA_ret,BND_ret,SPY_ret
count,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000
mean,132.007998,68.472209,334.256588,0.001828,0.000078,0.000575
std,120.917540,4.552961,126.413034,0.037285,0.003460,0.011491
min,9.578000,60.779140,155.869766,-0.210628,-0.054385,-0.109424
25%,18.972500,64.657953,230.428253,-0.017368,-0.001530,-0.003661
50%,94.977669,67.610096,305.298431,0.001261,0.000127,0.000645
75%,236.810833,71.891144,420.398643,0.019718,0.001783,0.005993
max,479.859985,77.318237,637.099976,0.226900,0.042201,0.105019


Saved:
 - ..\reports\figures\tsla_rolling_stats.png
 - ..\reports\interim\basic_stats.csv
ADF — TSLA (price): {'adf_stat': -1.419498605346799, 'p_value': 0.5728933715318626}
ADF — TSLA (log returns): {'adf_stat': -11.717126100741174, 'p_value': 1.442308894569334e-21}


,VaR_95_daily,Sharpe_annual,Annual_return_est,Annual_vol_est
TSLA,-0.054663,0.744550,0.460690,0.591887
BND,-0.004900,-0.007296,0.019599,0.054919
SPY,-0.017195,0.684427,0.144844,0.182407


Saved risk metrics CSV: ..\reports\interim\risk_metrics.csv
Artifacts saved:
 - ..\data\processed\merged_features.csv
 - ..\reports\figures\closing_prices.png
 - ..\reports\figures\daily_returns.png
 - ..\reports\figures\tsla_rolling_stats.png
 - ..\reports\interim\basic_stats.csv
 - ..\reports\interim\risk_metrics.csv
